# TP1
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 2.
Use o package Cryptography para criar uma cifra com autenticação de meta-dados a partir de um PRG
    1. Criar um gerador pseudo-aleatório do tipo XOF (“extened output function”) usando o SHAKE256, para gerar uma sequência de palavras de 64 bits. 
        1. O gerador deve poder gerar até um limite de $$\,2^n\,$$ palavras ($$n$$ é  um parâmetro) armazenados em long integers do Python.
        2. A “seed” do gerador funciona como $$\mathtt{cipher\_key}$$ e é gerado por um KDF a partir de uma “password” .
        3. A autenticação do criptograma e dos dados associados é feita usando o próprio SHAKE256.
    b. Defina os algoritmos de cifrar e decifrar : para cifrar/decifrar uma mensagem com blocos de 64 bits, os “outputs” do gerador são usados como máscaras XOR dos blocos da mensagem. 
    Essencialmente a cifra básica é uma implementação do  “One Time Pad”.

In [5]:
# Imports
import os
import time

from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, padding

# n -> parametro necessário para gerar as palavaras
N = 17
BLOCKSIZE = 8 # 64 bits = 8 bytes

In [13]:
def derivarChave(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=10000
    )
    chave = kdf.derive(password)
    return chave

In [14]:
def prg(seed):
    digest = hashes.Hash(hashes.SHAKE256(BLOCKSIZE * pow(2,N)))
    digest.update(seed)
    palavras = digest.finalize()
    return palavras

In [34]:
def cifrar(chave, mensagem):
    textoCifrado = b''
    padder = padding.PKCS7(64).padder()
    
    padded = padder.update(mensagem) + padder.finalize()
    
    mensagemBlocos = [padded[i:i + BLOCKSIZE] for i in range(0, len(padded), BLOCKSIZE)]
    
    for x in range(len(mensagemBlocos)):
        for indice, byte in enumerate(mensagemBlocos[x]):
            textoCifrado += bytes([byte ^ chave[x:(x+1) * BLOCKSIZE][indice]])
    
    return textoCifrado

In [35]:
def decifrar(chave, textoCifrado):
    plainText = b''
    
    mensagemBlocos = [textoCifrado[i:i + BLOCKSIZE] for i in range(0, len(textoCifrado), BLOCKSIZE)]
    
    for x in range(len(mensagemBlocos)):
        for indice, byte in enumerate(mensagemBlocos[x]):
            plainText += bytes([byte ^ chave[x:(x+1) * BLOCKSIZE][indice]])
    
    unpadder = padding.PKCS7(64).unpadder()
    
    unpadded = unpadder.update(plainText) + unpadder.finalize()
    
    return unpadded.decode("utf-8")

In [36]:
def main(): 
    password = "grupo17"
    
    salt = os.urandom(17)
    
    seed = derivarChave(password.encode("utf-8"), salt)
    
    chave = prg(seed)
    
    textoCifrado = cifrar(chave, "Estruturas Criptograficas".encode("utf-8"))

    print("Texto Cifrado: ")
    print(textoCifrado)
    print("")
    print("Plain Text: " + decifrar(chave, textoCifrado))

In [37]:
if __name__ == "__main__":
    main()

Texto Cifrado: 
b'.\xf6\xd9\xd2>\xbb\xb2\x1b\xe4\xde\x80\x08\xbd\xae\x19<\xc2\xc79\xae\xa1\x00+X\xd3L\xc8\xc0nO>A'

Plain Text: Estruturas Criptograficas
